In [18]:
!pip install beautifulsoup4
# !pip install openpyxl
!pip install pdfplumber
# !pip install gspread oauth2client

In [19]:
# import cell

import requests
from bs4 import BeautifulSoup
import pandas as pd

import pdfplumber
from io import BytesIO

import os
import json
import pdfplumber

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
file_path = "/content/drive/MyDrive/The Black Response- StopShotspotter /List of Websites for Web Scrapping.xlsx"

data = pd.read_excel(file_path)

website_urls = data['LINK'].tolist()

In [22]:
website_urls

['https://cambridgema.iqm2.com/Citizens/Calendar.aspx?From=1/1/2023&To=12/31/2023',
 'https://cambridgema.iqm2.com/Citizens/FileOpen.aspx?Type=1&ID=1176&Inline=True',
 'https://cambridgema.granicus.com/DocumentViewer.php?file=cambridgema_4f7397b2226656df692549693ff90ac4.pdf&view=1',
 'https://cambridgema.iqm2.com/Citizens/FileOpen.aspx?Type=1&ID=4027&Inline=True',
 'https://cambridgema.iqm2.com/Citizens/FileOpen.aspx?Type=1&ID=4036&Inline=True',
 'https://www.cambridgema.gov/Departments/cambridgepolice/News/2014/05/cambridgepolicedepartmenttoimplementshotspotterflexgunfirealertandanalysisservice',
 'https://cambridgema.iqm2.com/Citizens/FileOpen.aspx?Type=4&ID=10568&highlightTerms=shotspotter;',
 'https://cambridgema.iqm2.com/Citizens/FileOpen.aspx?Type=4&ID=9936&highlightTerms=shotspotter;',
 'https://cambridgema.iqm2.com/Citizens/Detail_Meeting.aspx?ID=4348',
 'https://cambridgema.iqm2.com/Citizens/FileOpen.aspx?Type=4&ID=8156&highlightTerms=shotspotter;',
 'https://cambridgema.iqm2.

In [23]:
key_terms = ["ShotSpotter", "Privacy", "Transparency", "Effective", "Reliability", "Urban areas", "Gentrification", "Inaccurate"]

In [24]:
folder_path = "/content/drive/MyDrive/The Black Response- StopShotspotter /Cambridge City Documents on ShotSpotter/Web Scraping PDF"

all_pdf_fnames = os.listdir(folder_path)

In [26]:
def pdfs_toprocess(all_pdfs, progress_file, output_path):
  path = os.path.join(output_path, progress_file)
  if os.path.exists(path):
    with open(path, 'r') as file:
      processed_pdfs = json.load(file)

  else:
    processed_pdfs = []

  todo_pdfs = [f for f in all_pdfs if f not in processed_pdfs]
  return todo_pdfs

def update_progress(pdf_name, progress_file, output_path):
  path = os.path.join(output_path, progress_file)
  if os.path.exists(path):
    with open(path, 'r') as file:
      processed_pdfs = json.load(file)
  else:
    processed_pdfs = []

  processed_pdfs.append(pdf_name)

  with open(path, 'w') as file:
    json.dump(processed_pdfs, file)

def update_result(pdf_name, result, result_file, output_path):
  path = os.path.join(output_path, result_file)
  if os.path.exists(path):
    with open(path, 'r') as file:
      results = json.load(file)
  else:
    results = {}

  results[pdf_name] = result

  with open(path, 'w') as file:
    json.dump(results, file)

def getKeyTermsCountPDF(pdf_fname, key_terms, source_path, output_path):
  pdf_path = os.path.join(source_path, pdf_fname)
  text = ''
  with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
      text += page.extract_text() or ''

  counts = {term: text.lower().count(term.lower()) for term in key_terms}
  return counts

In [27]:
source_folder_path = "/content/drive/MyDrive/The Black Response- StopShotspotter /Cambridge City Documents on ShotSpotter/Web Scraping PDF"
output_folder_path = "/content/drive/MyDrive/The Black Response- StopShotspotter /Web Scraping Results"

todo_pdfs = pdfs_toprocess(all_pdf_fnames, progress_file='progress.json', output_path=output_folder_path)
total_pdfs = len(todo_pdfs)

processed_pdf_cnt = 0

for fname in todo_pdfs:
  counts = getKeyTermsCountPDF(fname, key_terms, source_path=source_folder_path, output_path=output_folder_path)
  update_result(fname, counts, result_file='result.json', output_path=output_folder_path)
  update_progress(fname, progress_file='progress.json', output_path=output_folder_path)
  processed_pdf_cnt += 1

  percentage_done = processed_pdf_cnt/total_pdfs*100
  print(f'Successfully processed {fname}')
  print(f'Progress: {processed_pdf_cnt}/{total_pdfs} pdfs processed ({percentage_done:.2f}%)')

Successfully processed 2023-06-12 City Council - Public Minutes-2700.pdf
Progress: 1/11 pdfs processed (9.09)
Successfully processed 2020-02-24 City Council - Public Minutes-2165.pdf
Progress: 2/11 pdfs processed (18.18)
Successfully processed 2020-10-07 Public Safety Committee - Public Minutes-2235.pdf
Progress: 3/11 pdfs processed (27.27)
Successfully processed 2014-04-07 City Council - Full Agenda-1176.pdf
Progress: 4/11 pdfs processed (36.36)
Successfully processed 03-04-2024 (DocumentViewer).pdf
Progress: 5/11 pdfs processed (45.45)
Successfully processed Attachment-10568.pdf
Progress: 6/11 pdfs processed (54.55)
Successfully processed Attachment-9936.pdf
Progress: 7/11 pdfs processed (63.64)
Successfully processed 2024-03-18 City Council - Full Agenda-4036.pdf
Progress: 8/11 pdfs processed (72.73)
Successfully processed 2024-03-04 City Council - Full Agenda-4027.pdf
Progress: 9/11 pdfs processed (81.82)
Successfully processed 2023-05-23 Public Safety Committee - Full Agenda-3875.